In [12]:
import numpy as np
import random
import math
from utils import model

pi = 0.2
mu = 0.0
sigmabg = 0.001
sigma = 0.01
tau = 1 / (0.05 * 0.05)

x, y, csnps, v = model.simulate(pi = pi,
                             mu = mu,
                             sigmabg = sigmabg,
                             sigma = sigma,
                             tau = tau)

In [13]:
nhyp = 5
nvar = x.shape[0]
nsample = x.shape[1]
params = np.array([pi, mu, sigma, sigmabg, tau])
bounds = [[None, None] for i in range(nhyp)]
zstates = [[]]
zstates += [[i] for i in range(nvar)]

In [14]:
import time
start_time = time.time()

#def log_marginal_likelihood(params, x, y, zstates):
pi = params[0]
mu = params[1]
sigma = params[2]
sigmabg = params[3]
tau = params[4]
nvar = x.shape[0]
nsample = x.shape[1]
constpi = math.pi
kmarglik = 0
lmlzlist = list()
logk = 0

sigma2 = sigma * sigma
sigmabg2 = sigmabg * sigmabg

for z in zstates:
    nz = len(z)
    muz = np.zeros(nvar)
    muz[z] = mu
    #sigz = np.repeat(sigmabg2, nvar)
    #sigz[z] = sigma2
    
    sigz = np.array([sigmabg2] * nvar)
    sigz[z] = sigma2

    # P(z | theta)
    log_probz = nz * np.log(pi) + (nvar - nz) * np.log(1 - pi)

    # N(y | Mz, Sz)
    #Sz = np.einsum('ki, k, kj -> ij', x, sigz, x)
    Sz = np.dot(np.dot(x.T, np.diag(sigz)), x)
    Sz[np.diag_indices_from(Sz)] += 1/tau
    Mz = np.einsum('ij, i -> j', x, muz)
    logdetS = np.linalg.slogdet(Sz)[1]
    invS = np.linalg.inv(Sz)
    y_minus_M = y - Mz
    nterm = np.einsum('i, ij, j', y_minus_M, invS, y_minus_M)
    log_normz = - 0.5 * (logdetS + (nsample * np.log(2 * constpi)) + nterm)

    # Combine the log values
    if logk == 0: 
        logk = - log_probz - log_normz
    lmlz = log_probz + log_normz + logk
    kmarglik += np.exp(lmlz)
    lmlzlist.append(lmlz - logk)
logmarglik = - logk + np.log(kmarglik)
#    return lmlzlist, logmarglik

print("--- {:f} seconds ---\n".format(time.time() - start_time))
print("K * ML = {:e}, logML = {:f}\n".format(kmarglik, logmarglik))
#print("\n".join(["{:f}".format(x) for x in lmlzlist]))

--- 1.286062 seconds ---

K * ML = 2.607869e+12, logML = 436.387800



In [15]:
start_time = time.time()

#def log_marginal_likelihood(params, x, y, zstates):
pi = params[0]
mu = params[1]
sigma = params[2]
sigmabg = params[3]
tau = params[4]
nvar = x.shape[0]
nsample = x.shape[1]
constpi = math.pi
kmarglik = 0
lmlzlist = list()
logk = 0

sigma2 = sigma * sigma
sigmabg2 = sigmabg * sigmabg
h = 1/sigma2 - 1/sigmabg2

# calculate for zstate = [[]]
nz = 0
sigz0 = np.repeat(sigmabg2, nvar)

B0 = np.linalg.inv(np.diag(sigz0)) + tau * np.dot(x, x.T)
B0inv = np.linalg.inv(B0)
logB0det = np.linalg.slogdet(B0)[1]

log_probz = nz * np.log(pi) + (nvar - nz) * np.log(1 - pi)
#Sinv = - tau * tau * np.einsum('li, lk, kj -> ij', x, B0inv, x)
Sinv = - tau * tau * np.dot(np.dot(x.T, B0inv), x)
Sinv[np.diag_indices_from(Sinv)] += tau

logdetS = - nsample * np.log(tau) + (nvar - nz) * np.log(sigmabg2) + nz * np.log(sigma2) + logB0det

y_minus_M = y
nterm = np.einsum('i, ij, j', y_minus_M, Sinv, y_minus_M)
log_normz = - 0.5 * (logdetS + (nsample * np.log(2 * constpi)) + nterm)

logk = - log_probz - log_normz
lmlz = log_probz + log_normz + logk
kmarglik += np.exp(lmlz)
lmlzlist.append(lmlz - logk)

for z in zstates[1:]:
    nz = len(z)
    zpos = z[0]

    # P(z | theta)
    log_probz = nz * np.log(pi) + (nvar - nz) * np.log(1 - pi)

    # N(y | Mz, Sz)
    zpos = z[0]
    mod = h / (1 + h * B0inv[zpos, zpos])
    #BZinv = B0inv - mod * np.einsum('i, j -> ij', B0inv[:,zpos], B0inv[zpos,:])
    BZinv = B0inv - mod * np.outer(B0inv[:,zpos], B0inv[zpos,:])
    #Sinv = - tau * tau * np.einsum('li, lk, kj -> ij', x, BZinv, x)
    #Sinv = - tau * tau * np.einsum('ki, kj -> ij', x, np.einsum('ik,kj->ij', BZinv, x))
    Sinv = - tau * tau * np.dot(np.dot(x.T, BZinv), x)
    Sinv[np.diag_indices_from(Sinv)] += tau

    logBZdet = logB0det + np.log(1 + h * B0inv[zpos, zpos])
    logdetS = - nsample * np.log(tau) + (nvar - nz) * np.log(sigmabg2) + nz * np.log(sigma2) + logBZdet

    y_minus_M = y - mu * x[zpos, :]
    nterm = np.einsum('i, ij, j', y_minus_M, Sinv, y_minus_M)
    #nterm = np.dot(np.dot(y_minus_M.T, Sinv), y_minus_M)
    log_normz = - 0.5 * (logdetS + (nsample * np.log(2 * constpi)) + nterm)

    # Combine the log values
    lmlz = log_probz + log_normz + logk
    kmarglik += np.exp(lmlz)
    lmlzlist.append(lmlz - logk)
logmarglik = - logk + np.log(kmarglik)
#    return lmlzlist, logmarglik

print("--- {:f} seconds ---\n".format(time.time() - start_time))
print("K * ML = {:e}, logML = {:f}\n".format(kmarglik, logmarglik))
#print("\n".join(["{:f}".format(x) for x in lmlzlist]))

--- 0.402349 seconds ---

K * ML = 2.607869e+12, logML = 436.387800



In [16]:
[np.exp(x + logk) for x in lmlzlist]

[1.0,
 0.088166868102098547,
 0.07447144976080515,
 0.078905088244349192,
 0.072700897758548183,
 0.073861063001731675,
 0.13262337049034253,
 0.3463887284022632,
 0.69922270337186865,
 0.10290648826065292,
 0.091005861894412315,
 0.072127938948249995,
 0.071538886779160607,
 0.083578972320187864,
 59.700716662932606,
 0.083403119580906487,
 0.1279417237620242,
 0.093622322210104691,
 0.080006722897563678,
 0.076704319200621499,
 0.093664555631616242,
 0.067067527057745949,
 0.3267959609745717,
 0.069675346065658186,
 0.12388923842945675,
 0.066307550659077938,
 0.10187614976668756,
 0.078269737765206224,
 0.081468557119352261,
 0.094231030905015514,
 0.81221953034918992,
 0.077537837306608667,
 0.074653203737113774,
 0.076772085980978816,
 0.088490644233326379,
 0.2208548101000038,
 0.081403155542062247,
 0.076912699994770281,
 0.085916206935977929,
 0.11386716829765402,
 0.27253970247040926,
 0.079341736888863548,
 0.068536353226669552,
 0.067506038792666728,
 28274.517171282292,
 0.